# Building an AI Company Research Agent with Hyperbrowser and GPT-4o

In this cookbook, we'll build a Company Research Agent that can generate detailed reports on any company for any topic by automatically searching the web, analyzing search results, and compiling information from relevant sources.

We'll use these tools to build our agent:

- **[Hyperbrowser](https://hyperbrowser.ai)** for web search and reading web pages
- **OpenAI's GPT-4o** for intelligent analysis and report generation

By the end of this cookbook, you'll have a reusable agent that can research any company based on specific objectives, saving hours of manual research work!


## Prerequisites

To follow along you'll need the following:

1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai) if you don't have one, it's free)
2. An OpenAI API key (sign up at [openai.com](https://openai.com) if you don't have one, it's free)

Both API keys should be stored in a `.env` file in the same directory as this notebook with the following format:

```shell
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
OPENAI_API_KEY=your_openai_key_here
```


## Step 1: Set up imports and load environment variables


In [1]:
import os

from urllib.parse import urlencode

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models.extract import StartExtractJobParams
from hyperbrowser.models.scrape import ScrapeOptions, StartScrapeJobParams
from hyperbrowser.models.session import CreateSessionParams
from openai import AsyncOpenAI
from pydantic import BaseModel


import asyncio

load_dotenv()

True

## Step 2: Initialize clients


In [2]:
oai = AsyncOpenAI()
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))

## Step 3: Define search functionality

Now we'll create the models and function to search for information about a company. This function:

1. Constructs a search URL with the company name and research objective
2. Uses Hyperbrowser's extract feature to get structured data from search results
3. Returns the search results in a structured format using Pydantic models


In [3]:
class SearchResult(BaseModel):
    """A search result from Bing"""

    title: str
    url: str
    content: str

    def __str__(self):
        return f"Title: {self.title}\nURL: {self.url}\nContent: {self.content}"


class SearchResults(BaseModel):
    """A list of search results from Bing"""

    results: list[SearchResult]

    def __str__(self):
        return f"\n\n{'-' * 10}\n\n".join(str(result) for result in self.results)


async def search_company(company_name: str, objective: str) -> SearchResults:
    params = urlencode({"q": f"{company_name} {objective}"})
    url = f"https://www.bing.com/search?{params}"

    print(url)

    result = await hb.extract.start_and_wait(
        StartExtractJobParams(
            urls=[url],
            prompt="Extract the title, url, and content of the top 10 search results on this page.",
            schema=SearchResults,
        )
    )

    if not (result.status == "completed" and result.data):
        raise Exception("Failed to extract search results")

    return SearchResults.model_validate(result.data)

## Step 4: Choose relevant URLs with GPT-4o

After getting search results, we need to intelligently select the most relevant URLs to scrape. This function:

1. Uses GPT-4o to analyze the search results and choose the most relevant URLs
2. Applies a system prompt that guides the model to think step-by-step
3. Returns both the model's reasoning (chain of thought) and the selected URLs

You can also use a different re-ranker model here but for simplicity we've used GPT-4o.


In [4]:
class RelevantUrls(BaseModel):
    chain_of_thought: str
    urls: list[str]

    def __str__(self):
        return f"Chain of Thought: {self.chain_of_thought}\n\nURLs:\n{self.urls}"


RELEVANT_URLS_SYSTEM_PROMPT = """
You are helping analyze search results to find the most relevant URLs for researching a company. \
Think step by step about which of the URLs you are given would be most useful considering the \
objective and the company name. Always respond with both your chain of thought and the list of \
relevant URLs.""".strip()


async def choose_relevant_urls(
    company_name: str, objective: str, search_results: SearchResults
) -> RelevantUrls:
    response = await oai.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": RELEVANT_URLS_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": (
                    f"Company: {company_name}\nResearch Objective: {objective}"
                    f"\n\nSearch Results:\n{search_results}"
                    "Think through which URLs would be most relevant and explain your reasoning. Return your analysis and final list of URLs along with your chain of thought."
                ),
            },
        ],
        response_format=RelevantUrls,
    )

    urls = response.choices[0].message.parsed
    if urls is None:
        raise Exception("Could not select relevant urls")
    return urls

## Step 5: Scrape content from selected URLs

Once we have the most relevant URLs, we need to scrape their content. This function:

1. Uses Hyperbrowser's batch scraping capability to process multiple URLs efficiently
2. Configures advanced scraping options like proxy usage, stealth mode, and CAPTCHA solving
3. Returns the scraped content in a structured format for further processing


In [5]:
from hyperbrowser.models.scrape import ScrapeJobResponse


async def scrape_url(url: str):
    scrape_result = await hb.scrape.start_and_wait(
        StartScrapeJobParams(
            url=url,
            scrape_options=ScrapeOptions(formats=["markdown"], only_main_content=True),
            session_options=CreateSessionParams(
                use_proxy=True,
                use_stealth=True,
                adblock=True,
                trackers=True,
                annoyances=True,
                solve_captchas=True,
            ),
        )
    )
    return (url, scrape_result)

def limit_content_for_tokens(scrape_results, max_total_chars=20000):
    """
    Intelligently limit scraped content to avoid token limits.
    
    Args:
        scrape_results: List of (url, page) tuples from scraping
        max_total_chars: Maximum total characters to allow (default 20000)
        
    Returns:
        String with formatted website content within token limits
    """
    result_texts = []
    total_chars = 0
    
    # First pass: collect all valid results
    valid_results = [(url, page.data.markdown) 
                    for url, page in scrape_results 
                    if page.data is not None and page.data.markdown is not None]
    
    if valid_results:
        # Calculate chars per site based on available space
        chars_per_site = max_total_chars // len(valid_results)
        
        # Second pass: add truncated content
        for url, content in valid_results:
            if total_chars >= max_total_chars:
                break
                
            # Truncate if needed
            if len(content) > chars_per_site:
                content = content[:chars_per_site] + "..."
            
            result_text = f"<website>\n<url>{url}</url>\n<content>\n{content}\n</content>\n</website>"
            result_texts.append(result_text)
            total_chars += len(result_text)
    
    return "\n".join(result_texts)
           

async def scrape_urls(urls: list[str]) -> str:
    try:
        scrape_requests = [scrape_url(url) for url in urls]
        scrape_results = await asyncio.gather(*scrape_requests)

        # If this causes you to run into rate limits, then use this instead
        # return limit_content_for_tokens(scrape_results) 

        return "\n".join(
            [
                f"<website>\n<url>{url}</url>\n<content>\n{page.data.markdown}\n</content>\n</website>"
                for url, page in scrape_results
                if page.data is not None and page.data.markdown is not None
            ]
        )
       
        
    except Exception as e:
        raise Exception("Failed to scrape URLS")

## Step 6: Generate research report with GPT-4o

With the scraped content in hand, we can now generate a comprehensive research report. This function:

1. Uses GPT-4o with a specialized system prompt for research analysis
2. Provides the model with the company name, research objective, and scraped data
3. Returns both the model's reasoning process and the final formatted report


In [6]:
class ResearchAnalysis(BaseModel):
    chain_of_thought: str
    report: str

    def __str__(self):
        return f"Chain of Thought:\n{self.chain_of_thought}\n\nReport:\n{self.report}"


FINAL_REPORT_SYSTEM_PROMPT = """
You are an expert research analyst that compiles information from a list of websites into a report about a company.

You will be given the following information:
- Company Name
- Research Objective
- Scraped Data from the most relevant webpages about the company as it relates to the research objective / topic

Your job is to compile the information into a report that is easy to understand and can be used to make decisions about the company.

The report should be:
- In markdown format
- Concise and to the point
- Include the company name, objective, and the information found on the websites
- Be easy to understand and can be used to make decisions about the company

Additionally, you should also maintain a scratchpad for your own notes and thoughts about the company as you draft the report.

You must respond with both your chain of thought and the final report.""".strip()


async def compile_into_report(
    scraped_data: str, company_name: str, objective: str
) -> ResearchAnalysis:
    response = await oai.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": FINAL_REPORT_SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": (
                    f"Company Name: {company_name}\nResearch Objective: {objective}"
                    f"\nScraped Websites / Articles: {scraped_data}"
                ),
            },
        ],
        response_format=ResearchAnalysis,
    )

    analysis = response.choices[0].message.parsed
    if (analysis is None):
        raise Exception(f"Could not get analysis for {company_name}")

    return analysis

## Step 7: Combine all steps into a single research pipeline

Now we'll create a single function that orchestrates the entire research process from start to finish. This function:

1. Searches for information about the company
2. Selects the most relevant URLs
3. Scrapes content from those URLs
4. Generates a comprehensive research report
5. Returns the final analysis


In [7]:
async def research_company(company_name: str, objective: str) -> ResearchAnalysis:
    search_results = await search_company(company_name, objective)
    relevant_urls = await choose_relevant_urls(company_name, objective, search_results)
    scraped_data = await scrape_urls(relevant_urls.urls)
    analysis = await compile_into_report(scraped_data, company_name, objective)
    return analysis

## Step 8: Test the research agent

Let's test our agent by researching UiPath's competitive landscape. This will demonstrate the full workflow:

1. The agent searches for information about UiPath's competition
2. It selects the most relevant URLs from the search results
3. It scrapes content from those URLs
4. It generates a comprehensive research report

You'll see the search URL being printed, followed by the final analysis that includes both the reasoning process and the formatted report.


In [8]:
analysis = await research_company("UiPath", "Competition")
print(analysis)

https://www.bing.com/search?q=UiPath+Competition
Chain of Thought:
With UiPath as one of the leading names in the robotic process automation (RPA) industry, it faces significant competition from various competitors that offer both alternative and complementary technology solutions. Companies like Automation Anywhere, Blue Prism, and Microsoft Power Automate are direct RPA competitors, each offering unique benefits that challenge UiPath's dominance. Zluri and WorkFusion also provide distinctive capabilities like cloud-native platforms and AI integration, respectively. Other industry entrants, such as IBM, Pega, and NICE, present robust RPA solutions tailored to specific industry needs, highlighting important comparative elements like ease of integration, scalability, and pricing. Evaluating UiPath against key competitors reveals both strategic opportunities and challenges due to the rapidly evolving landscape of enterprise automation technologies.

Report:
# Company Report: UiPath

## *

## Step 9: Try it with your own research topics

Now that you've seen how the agent works, you can try researching different companies or objectives. Simply modify the code below with your desired company name and research objective.

```python
# Example: Research a different company's product strategy
# analysis = await research_company("Stripe", "Product Strategy")
# print(analysis)
```

Feel free to experiment with different research topics such as:

- Company financials
- Market positioning
- Leadership team
- Recent news and developments
- Industry trends


## Conclusion

In this cookbook, we built a powerful company research agent using Hyperbrowser and OpenAI's GPT-4o. This agent can:

1. Automatically search for information about any company
2. Intelligently select the most relevant sources
3. Extract and process content from those sources
4. Generate comprehensive, well-structured research reports
5. Save hours of manual research work

This pattern can be extended to create more sophisticated research agents for different domains, use additional data sources, or be integrated into larger applications.

### Next Steps

To take this further, you might consider:

- Adding more specialized research objectives (financial analysis, competitive analysis, etc.)
- Implementing caching to improve performance for repeated queries
- Creating a web interface for easier interaction
- Adding support for additional data sources like financial databases or social media
- Implementing a feedback loop to improve the quality of reports over time

Happy Building! 😃


## Relevant Links

- [Hyperbrowser](https://hyperbrowser.ai)
- [Hyperbrowser Documentation](https://docs.hyperbrowser.ai)
- [OpenAI API Documentation](https://platform.openai.com/docs/introduction)
- [Pydantic Documentation](https://docs.pydantic.dev/)
